In [ ]:
!pip install accelerate
!pip install git+https://github.com/huggingface/diffusers

In [ ]:
!pip install fastapi uvicorn pyngrok

In [ ]:
from diffusers import DiffusionPipeline, UNet2DConditionModel
from fastapi.responses import JSONResponse
from google.colab import userdata
from pydantic import BaseModel
from fastapi import FastAPI
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import base64
import io

In [ ]:
class ImageGenerationModel():
    """
    A class for generating images using the Stable Diffusion model.

    Attributes:
        unet: UNet model for image processing.
        pipeline: Diffusion pipeline for generating images using the diffusion model.

    Methods:
        get_generated_image(): Generates an image based on the given text prompt.
    """

    def __init__(self):
        """
        Initializes the model by loading the pretrained UNet model and setting up the pipeline for image generation.
        """
        self.unet = UNet2DConditionModel.from_pretrained("shshshamilll/unet")
        self.pipeline = DiffusionPipeline.from_pretrained(
            "stable-diffusion-v1-5/stable-diffusion-v1-5", unet=self.unet
        ).to("cuda")
        self.pipeline.safety_checker = None

    def get_generated_image(self, prompt):
        """
        Generates an image based on the given text prompt.

        Parameters:
            prompt: The text prompt for generating the image.

        Returns:
            The generated image.
        """
        generated_image = self.pipeline(
            prompt,
            negative_prompt="Shirt sleeves, black shirt, black background",
            guidance_scale=8.5,
            num_inference_steps=100
        ).images[0]
        return generated_image

In [ ]:
ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
userdata.get('HF_TOKEN')

ngrok.set_auth_token(ngrok_authtoken)
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url.public_url}")

In [ ]:
app = FastAPI()
image_generation_model = ImageGenerationModel()

In [ ]:
class ImageGenerationRequest(BaseModel):
    """
    Data model for an image generation request.

    Attributes:
        prompt: A prompt.
    """

    prompt: str

def encode_image(image):
    """
    Encodes an image to base64 format.

    Parameters:
        image: The image to be encoded.

    Returns:
        The image in base64 format.
    """
    image_buffer = io.BytesIO()
    image.save(image_buffer, format="PNG")
    return base64.b64encode(image_buffer.getvalue()).decode("utf-8")

@app.post("/get_remote_generated_image")
def get_remote_generated_image(image_generation_request: ImageGenerationRequest):
    """
    Generates an image on a remote server based on the request and returns it in base64 format.

    This function handles the POST request with a prompt, generates the image, and returns it in base64 format.

    Parameters:
        image_generation_request: The object containing the prompt for image generation.

    Returns:
        A response containing the image in base64 format.
    """
    remote_generated_image = image_generation_model.get_generated_image(image_generation_request.prompt)
    return JSONResponse(content={"remote_generated_image_in_base64_format": encode_image(remote_generated_image)})

In [ ]:
nest_asyncio.apply()
uvicorn.run(app, port=8000)